# **💳 카드 상품 추천 대화형AI 구현 프로젝트(체크카드)**
1. CSV파일 로드, Document객체로 만들기
2. 청킹
3. 임베딩
4. VectorDB에 저장
5. 검색
6. 프롬프트 작성
7. LLM모델 객체 생성

In [1]:
!pip install -U langchain-community

In [2]:
!pip install langchain-openai

In [3]:
from openai import OpenAI
from getpass import getpass
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re

from langchain_openai import OpenAIEmbeddings

## 1. CSV 파일 로드, Document 객체 만들기
- CSVLoader 사용

In [14]:
import pandas as pd

In [59]:
df = pd.read_csv(r"C:\Users\82108\study seoan\data\debit_cards.csv")
print(df.head())

          card_name card_corp  \
0          ONE 체크카드      케이뱅크   
1  노리2 체크카드(KB Pay)    KB국민카드   
2        KB 틴업 체크카드    KB국민카드   
3           모빌리언스카드   KG모빌리언스   
4          더나은 체크카드   MG새마을금고   

                                            benefits  
0  최대 1.1% 무제한 캐시백 자주 쓰는 곳에서 5% 캐시백 3번 결제할 때마다 1,...  
1   커피, 모바일, 문화 10% 할인 뷰티, 편의점 5% 할인 KB Pay 2% 추가 할인  
2           편의점 5% 할인 다이소, 올리브영 5% 할인 PC방, 모바일 5% 할인  
3  온라인/오프라인 각각 1%/0.5% 적립 편의점, 커피, 외식 최대 8% 적립 KF...  
4              간편결제 20% 할인 온라인쇼핑 20% 할인 커피전문점 20% 할인  


In [16]:
loader = CSVLoader('data/debit_cards.csv', encoding = 'utf-8-sig')
doc = loader.load()
doc

[Document(metadata={'source': 'data/debit_cards.csv', 'row': 0}, page_content='card_name: ONE 체크카드\ncard_corp: 케이뱅크\nbenefits: 최대 1.1% 무제한 캐시백 자주 쓰는 곳에서 5% 캐시백 3번 결제할 때마다 1,000원 무제한 캐시백'),
 Document(metadata={'source': 'data/debit_cards.csv', 'row': 1}, page_content='card_name: 노리2 체크카드(KB Pay)\ncard_corp: KB국민카드\nbenefits: 커피, 모바일, 문화 10% 할인 뷰티, 편의점 5% 할인 KB Pay 2% 추가 할인'),
 Document(metadata={'source': 'data/debit_cards.csv', 'row': 2}, page_content='card_name: KB 틴업 체크카드\ncard_corp: KB국민카드\nbenefits: 편의점 5% 할인 다이소, 올리브영 5% 할인 PC방, 모바일 5% 할인'),
 Document(metadata={'source': 'data/debit_cards.csv', 'row': 3}, page_content='card_name: 모빌리언스카드\ncard_corp: KG모빌리언스\nbenefits: 온라인/오프라인 각각 1%/0.5% 적립 편의점, 커피, 외식 최대 8% 적립 KFC 10% 즉시할인'),
 Document(metadata={'source': 'data/debit_cards.csv', 'row': 4}, page_content='card_name: 더나은 체크카드\ncard_corp: MG새마을금고\nbenefits: 간편결제 20% 할인 온라인쇼핑 20% 할인 커피전문점 20% 할인'),
 Document(metadata={'source': 'data/debit_cards.csv', 'row': 5}, page_content='card_nam

## 2. 청킹
- RecursiveCharacterTextSplitter 사용

In [17]:
# splitter 객체 생성
csv_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,
                                             chunk_overlap = 50,
                                             separators = ["\n\n", "\n", " "])

In [18]:
# loader와 splitter 실제로 적용
my_chunks = [] # CSV에서 잘라낸 청크들을 보관할 리스트

# load_and_split함수를 사용하면 반환되는 형식이 리스트이므로 extend 함수를 사용
my_chunks.extend(loader.load_and_split(csv_splitter))
print("number of chunks: ", len(my_chunks))
print(my_chunks)

number of chunks:  364
[Document(metadata={'source': 'data/debit_cards.csv', 'row': 0}, page_content='card_name: ONE 체크카드\ncard_corp: 케이뱅크\nbenefits: 최대 1.1% 무제한 캐시백 자주 쓰는 곳에서 5% 캐시백 3번 결제할 때마다 1,000원 무제한 캐시백'), Document(metadata={'source': 'data/debit_cards.csv', 'row': 1}, page_content='card_name: 노리2 체크카드(KB Pay)\ncard_corp: KB국민카드\nbenefits: 커피, 모바일, 문화 10% 할인 뷰티, 편의점 5% 할인 KB Pay 2% 추가 할인'), Document(metadata={'source': 'data/debit_cards.csv', 'row': 2}, page_content='card_name: KB 틴업 체크카드\ncard_corp: KB국민카드\nbenefits: 편의점 5% 할인 다이소, 올리브영 5% 할인 PC방, 모바일 5% 할인'), Document(metadata={'source': 'data/debit_cards.csv', 'row': 3}, page_content='card_name: 모빌리언스카드\ncard_corp: KG모빌리언스\nbenefits: 온라인/오프라인 각각 1%/0.5% 적립 편의점, 커피, 외식 최대 8% 적립 KFC 10% 즉시할인'), Document(metadata={'source': 'data/debit_cards.csv', 'row': 4}, page_content='card_name: 더나은 체크카드\ncard_corp: MG새마을금고\nbenefits: 간편결제 20% 할인 온라인쇼핑 20% 할인 커피전문점 20% 할인'), Document(metadata={'source': 'data/debit_cards.csv', 'row': 5}, page

## 체크카드 개수랑 동일한 364개의 chunk로 분리됨.
## ➡️ 청킹할 필요가 없더라!!^^;;

## 3. 임베딩
- 문장(텍스트)을 **숫자 벡터(=리스트)**로 변환해주는 역할\
👉 AI가 “의미”를 수학적으로 이해할 수 있게 만드는 과정!
- 임베딩 모델 API를 사용하여 임베딩 객체 생성

In [96]:
MY_API_KEY = getpass.getpass("OpenAI API Key : ")

OpenAI API Key :  ········


In [97]:
client = OpenAI(api_key = MY_API_KEY)

In [23]:
# 임베딩
my_embedding = OpenAIEmbeddings(model = "text-embedding-3-small",
                               api_key=MY_API_KEY)
my_embedding

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x00000249C7C18830>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x00000249C6E87A10>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

## 4. Chroma(VectorDB)에 저장
- Chroma: **벡터 DB(데이터베이스)**로, 임베딩된 문서를 저장하고 검색할 수 있게 해줌.
- .from_documents(): 문서들을 임베딩해서 ChromaDB에 바로 넣어주는 함수\
 👉 결과적으로 vectordb 안에는 PDF → 청크 → 벡터화 → 벡터DB 저장 과정이 다 끝난 상태

In [24]:
# langchain연동 chroma DB 설치
!pip install langchain-chroma

In [25]:
from langchain_chroma import Chroma

In [26]:
my_directory = "/VectorStores" # DB가 저장될 폴더 경로를 지정

vectordb = Chroma.from_documents(documents = my_chunks, # 벡터화 시킬 청크 목록
                                embedding = my_embedding, # 임베딩 객체
                                persist_directory = my_directory) # 벡터 DB 저장 경로

In [27]:
# 수집된 문서(청크) 개수 확인
vectordb._collection.count()
# 체크카드 개수인 364개가 잘 나옴.

377

## 5. 유사도 검색
- 간단한 질문을 해서 유사한 문서 검색
- similarity_search()함수 사용
  - 입력한 질문을 임베딩으로 바꾸고 vectordb 안에 저장된 모든 청크와 비교해서, 코사인 유사도(cosine similarity)가 가장 높은 것들을 뽑아냄.
  - k값: 제일 비슷한 문서 k개를 가져와라! 라는 뜻

In [28]:
question = "최대 1.1% 무제한 캐시백해주고, 자주 쓰는 곳에서 5% 캐시백해주고, 3번 결제할 때마다 1,000원 무제한 캐시백해주는 혜택이 있는 카드가 뭐야?"
similar_docs = vectordb.similarity_search(question, k=3)
print("객체 수 : ", len(similar_docs)) # k값이 3이니까 객체 수는 3임.
print()
similar_docs

객체 수 :  3



[Document(id='b1c26a84-275b-4239-bcd9-9d7270c63af0', metadata={'row': 0, 'source': 'data/debit_cards.csv'}, page_content='card_name: ONE 체크카드\ncard_corp: 케이뱅크\nbenefits: 최대 1.1% 무제한 캐시백 자주 쓰는 곳에서 5% 캐시백 3번 결제할 때마다 1,000원 무제한 캐시백'),
 Document(id='a968ccbc-fd84-47c0-8d89-11639bdfad91', metadata={'source': 'data/debit_cards.csv', 'row': 243}, page_content='card_name: 하나로 전자카드 메가캐쉬백 더드림 체크카드\ncard_corp: 하나카드\nbenefits: 생활필수업종 2만원당 200원 캐시백 통신료 최대 5천원 캐시백 CU편의점 포인트 2% 적립'),
 Document(id='ea649406-7bc4-4f18-88b7-c5c7038397c1', metadata={'row': 6, 'source': 'data/debit_cards.csv'}, page_content='card_name: 토스뱅크 체크카드\ncard_corp: 토스뱅크\nbenefits: 일상 속 7가지 영역 월 최대 35,000원 캐시백 온라인 결제 7가지 영역 월 최대 14,000원 캐시백 국내 온·오프라인 어디서나 0.3% 캐시백')]

## 5. MMR(Maximal Marginal Relevance) 유사도 검색
**"나랑 비슷하면서도 서로 다른 애들 골라줘!"**
- 🔎 일반 유사도 검색 (similarity_search)
  - 질문(query)랑 가장 비슷한 문서들을 k개 뽑아옴.
  - 내용이 겹치는 문서만 우르르 나올 수 있음 → 다양성이 없음.\
  👉 질문: "기술평가란 뭐야?"\
  👉 결과: "기술평가 개요", "기술평가 개요", "기술평가 개요" (비슷한 내용만 반복)
- 🔎 MMR (Maximal Marginal Relevance) 검색
  - similarity_search()보다 똑똑하게 검색하는 방법
  - 검색 결과가 질문과 비슷한 정도 + 서로 다른 정도(다양성) 둘 다 고려
  - 비슷하면서도 중복 안 되는 결과를 골라주는 알고리즘

In [29]:
# fetch_k=6 → 일단 6개 뽑아보고
# k=3 → 그중에서 최종적으로 3개만 선택
search_mmr = vectordb.max_marginal_relevance_search(question, fetch_k=6, k=3, lambda_mult=0.5)
search_mmr

[Document(id='b1c26a84-275b-4239-bcd9-9d7270c63af0', metadata={'row': 0, 'source': 'data/debit_cards.csv'}, page_content='card_name: ONE 체크카드\ncard_corp: 케이뱅크\nbenefits: 최대 1.1% 무제한 캐시백 자주 쓰는 곳에서 5% 캐시백 3번 결제할 때마다 1,000원 무제한 캐시백'),
 Document(id='3a7c2cec-4eb6-40a7-b51e-20d15277e6ba', metadata={'row': 241, 'source': 'data/debit_cards.csv'}, page_content='card_name: 신한카드 Hey Young 체크(잔망루피)\ncard_corp: 신한카드\nbenefits: 일상 서비스 최대 5천원 캐시백 간편결제 서비스 최대 5천원 캐시백 해외 이용서비스 1.2% 캐시백'),
 Document(id='7b8ab1e5-5f0a-4b4e-9108-3f4a7574d868', metadata={'source': 'data/debit_cards.csv', 'row': 325}, page_content='card_name: 핀크카드\ncard_corp: 핀크카드\nbenefits: 이용실적에 따라 최대1% 적립 연말정산 30% 공제')]

## 6. .as_retriever()함수
**벡터DB를 LangChain 전용 “검색기”로 바꿔주는 함수**
- vectordb 같은 벡터 DB는 그냥 "문서 저장소 + 검색 도구"인데, 이걸 **LangChain에서 쓰는 검색기(retriever)**로 변환해야,
다른 기능(RetrievalQA, ConversationalRetrievalChain 같은 거)과 딱딱 맞게 붙을 수 있음.

In [30]:
my_retriever = vectordb.as_retriever(search_kwargs={"k":3}) # 검색할 유사도가 높은 청크 3개 가져오라는 옵션 (디폴트는 4개)

In [31]:
question = "최대 1.1% 무제한 캐시백해주고, 자주 쓰는 곳에서 5% 캐시백해주고, 3번 결제할 때마다 1,000원 무제한 캐시백해주는 혜택이 있는 카드가 뭐야?"

relevant_docs = my_retriever.invoke(question)
# my_retriever가 벡터DB에서 질문과 제일 가까운 문서 청크 3개를 찾아서 돌려줌.
relevant_docs

[Document(id='b1c26a84-275b-4239-bcd9-9d7270c63af0', metadata={'row': 0, 'source': 'data/debit_cards.csv'}, page_content='card_name: ONE 체크카드\ncard_corp: 케이뱅크\nbenefits: 최대 1.1% 무제한 캐시백 자주 쓰는 곳에서 5% 캐시백 3번 결제할 때마다 1,000원 무제한 캐시백'),
 Document(id='a968ccbc-fd84-47c0-8d89-11639bdfad91', metadata={'source': 'data/debit_cards.csv', 'row': 243}, page_content='card_name: 하나로 전자카드 메가캐쉬백 더드림 체크카드\ncard_corp: 하나카드\nbenefits: 생활필수업종 2만원당 200원 캐시백 통신료 최대 5천원 캐시백 CU편의점 포인트 2% 적립'),
 Document(id='ea649406-7bc4-4f18-88b7-c5c7038397c1', metadata={'row': 6, 'source': 'data/debit_cards.csv'}, page_content='card_name: 토스뱅크 체크카드\ncard_corp: 토스뱅크\nbenefits: 일상 속 7가지 영역 월 최대 35,000원 캐시백 온라인 결제 7가지 영역 월 최대 14,000원 캐시백 국내 온·오프라인 어디서나 0.3% 캐시백')]

In [32]:
# 검색기에 MMR검색도 적용 가능
# fetch_k=6 → 일단 6개 뽑아보고
# k=3 → 그중에서 최종적으로 3개만 선택
my_retriever_mmr = vectordb.as_retriever(search_type="mmr",
                                         search_kwargs={"fetch_k":6, "k":3, "lambda_mult":0.5}
                                        )

In [33]:
relevant_docs_mmr = my_retriever_mmr.invoke(question)
relevant_docs_mmr

[Document(id='b1c26a84-275b-4239-bcd9-9d7270c63af0', metadata={'row': 0, 'source': 'data/debit_cards.csv'}, page_content='card_name: ONE 체크카드\ncard_corp: 케이뱅크\nbenefits: 최대 1.1% 무제한 캐시백 자주 쓰는 곳에서 5% 캐시백 3번 결제할 때마다 1,000원 무제한 캐시백'),
 Document(id='3a7c2cec-4eb6-40a7-b51e-20d15277e6ba', metadata={'row': 241, 'source': 'data/debit_cards.csv'}, page_content='card_name: 신한카드 Hey Young 체크(잔망루피)\ncard_corp: 신한카드\nbenefits: 일상 서비스 최대 5천원 캐시백 간편결제 서비스 최대 5천원 캐시백 해외 이용서비스 1.2% 캐시백'),
 Document(id='7b8ab1e5-5f0a-4b4e-9108-3f4a7574d868', metadata={'source': 'data/debit_cards.csv', 'row': 325}, page_content='card_name: 핀크카드\ncard_corp: 핀크카드\nbenefits: 이용실적에 따라 최대1% 적립 연말정산 30% 공제')]

## 7. RetrievalQA
- 질의응답(QA) 전용 클래스
- 내부에서 두 가지를 합쳐줌.
  - Retriever (벡터DB 검색기 = 관련 문서 찾기)
  - LLM (GPT 같은 모델 = 문서 읽고 답 생성하기)
➡️ 문서 검색 + 답변 생성을 한 번에 처리해줌!!

In [34]:
!pip install pydantic

In [35]:
from langchain.chains import RetrievalQA
# langchain과 연동된 OpenAI Chat모델을 활용
from langchain_openai import ChatOpenAI

In [36]:
# LLM 생성
chat_model = ChatOpenAI(model='gpt-3.5-turbo',
                        api_key=MY_API_KEY
                       )

In [37]:
R_QA1 = RetrievalQA.from_chain_type(llm=chat_model,               # 답변해줄 LLM 지정
                                    chain_type="refine",          # 문서 처리 방식 선택
                                    retriever=my_retriever,       # 벡터DB 검색기
                                    return_source_documents=True  # 어떤 내용을 참조했는지 출력
                                   )

In [38]:
question = "최대 1.1% 무제한 캐시백해주고, 자주 쓰는 곳에서 5% 캐시백해주고, 3번 결제할 때마다 1,000원 무제한 캐시백해주는 혜택이 있는 카드가 뭐야?"
llm_response = R_QA1.invoke(question) # "검색 + 답변"이 한 번에 실행됨.
llm_response

{'query': '최대 1.1% 무제한 캐시백해주고, 자주 쓰는 곳에서 5% 캐시백해주고, 3번 결제할 때마다 1,000원 무제한 캐시백해주는 혜택이 있는 카드가 뭐야?',
 'result': '원래 답변을 유지합니다. 토스뱅크 체크카드는 최대 1.1% 무제한 캐시백이 아닌, 일상 속 7가지 영역에 대해 월 최대 35,000원 캐시백, 온라인 결제 7가지 영역에 대해 월 최대 14,000원 캐시백, 그리고 국내 온·오프라인 어디서나 0.3% 캐시백 혜택을 제공하는 카드입니다.',
 'source_documents': [Document(id='b1c26a84-275b-4239-bcd9-9d7270c63af0', metadata={'source': 'data/debit_cards.csv', 'row': 0}, page_content='card_name: ONE 체크카드\ncard_corp: 케이뱅크\nbenefits: 최대 1.1% 무제한 캐시백 자주 쓰는 곳에서 5% 캐시백 3번 결제할 때마다 1,000원 무제한 캐시백'),
  Document(id='a968ccbc-fd84-47c0-8d89-11639bdfad91', metadata={'row': 243, 'source': 'data/debit_cards.csv'}, page_content='card_name: 하나로 전자카드 메가캐쉬백 더드림 체크카드\ncard_corp: 하나카드\nbenefits: 생활필수업종 2만원당 200원 캐시백 통신료 최대 5천원 캐시백 CU편의점 포인트 2% 적립'),
  Document(id='ea649406-7bc4-4f18-88b7-c5c7038397c1', metadata={'source': 'data/debit_cards.csv', 'row': 6}, page_content='card_name: 토스뱅크 체크카드\ncard_corp: 토스뱅크\nbenefits: 일상 속 7가지 영역 월 최대 35,000원 캐시백 온라인 결제 7가지 영역 월 최대 14,000원 캐시백 

In [39]:
print(llm_response['query']) # 질문 출력
print(llm_response['result']) # Retriever가 찾아낸 문서 조각들을 기반으로 LLM이 생성한 답변

최대 1.1% 무제한 캐시백해주고, 자주 쓰는 곳에서 5% 캐시백해주고, 3번 결제할 때마다 1,000원 무제한 캐시백해주는 혜택이 있는 카드가 뭐야?
원래 답변을 유지합니다. 토스뱅크 체크카드는 최대 1.1% 무제한 캐시백이 아닌, 일상 속 7가지 영역에 대해 월 최대 35,000원 캐시백, 온라인 결제 7가지 영역에 대해 월 최대 14,000원 캐시백, 그리고 국내 온·오프라인 어디서나 0.3% 캐시백 혜택을 제공하는 카드입니다.


## 8. Reranking
**“검색된 문서들 중에서 정답과 더 관련 있는 순서”로 재정렬해주는 도구**
- 벡터DB(similarity_search나 MMR)가 질문과 유사한 문서를 가져오긴 하지만...
  - 꼭 중요한 문서만 가져온다는 보장은 없음.
  - "엉뚱하게 비슷한데 덜 중요한 문서"가 앞에 나올 수도 있음.

👉 이때 **Reranking(재정렬)**을 쓰면 답변 품질이 올라감.
- **CohereRerank** 클래스
  - 질문(쿼리)과 문서들을 비교해서, 답변에 가장 도움이 되는 문서 순서로 재배열해줌.
  - 유사도 검색만 쓰는 것보다 더 정확한 검색 결과를 얻을 수 있음.


In [40]:
# langchain과 연동된 cohere 라이브러리 설치
!pip install langchain-cohere

In [41]:
from langchain_cohere import CohereRerank

In [82]:
COHERE_API_KEY = getpass.getpass("Cohere API Key :")

Cohere API Key : ········


In [83]:
# CohereRerank객체 생성
my_rerank = CohereRerank(cohere_api_key=COHERE_API_KEY,
                         model="rerank-v3.5",   # cohere에서 지원하는 reranking모델
                         top_n=3                # rerank모델이 가장 관련성이 높은 3개의 결과만 반환
                        )

## 9. 문맥 압축
- 가장 관련성이 높은 부분을 선택하고, 덜 중요한 정보는 삭제하여 핵심만 남겨두는 것
### ContextualCompressionRetriever클래스
- 검색된 문서들 중에서 덜 중요한 부분은 잘라내고, 핵심만 남겨주는 검색기
- base_retriever: 벡터DB에서 후보 문서 뽑아옴
- base_compressor: 그 후보들을 추려내거나 요약

In [84]:
from langchain.retrievers import ContextualCompressionRetriever

In [85]:
compression_retriever = ContextualCompressionRetriever(base_compressor=my_rerank,
                                                       base_retriever=my_retriever
                                                      )

In [86]:
R_QA2 = RetrievalQA.from_chain_type(
    llm=chat_model,
    chain_type="refine",
    retriever=my_retriever  # CohereRerank 제거
)

## 10. LLM모델 사용하기

In [87]:
prompt = """ 
너는 고객들에게 딱 맞는 체크카드를 추천해주는 경력 20년차 은행원이야.
고객의 프로필을 보고 딱 맞는 체크카드 3개를 추천해주고 추천하는 이유를 알려주는 게 너의 일이야.
"""

In [88]:
question = """
25세 여성 / 서울 서초구 거주(원룸 월세) / 1인 가구, 햄스터 1마리 / 스타트업 마케팅팀 사원(입사 1년 차), 블로거 
/ 아침 거름, 점심은 회사 근처 식당, 저녁은 배달·간편식·외식 위주 / 세후 월 소득 230만 원 / 예금 2000만 원, 적금 월 30만 원 / 자동차 없음(대중교통 이용) 
/ 영양제 자주 구매, 운동 자주 하지만 술 많이 마심 / 취미: 브이로그 촬영, 전시회·팝업스토어·카페 탐방, 프리다이빙, 스쿠버다이빙 / 
지출: 절반 이상 취미활동, 나머지는 월세·햄스터 용품 / 특이사항: 외식 협찬 블로그 운영, 집에서 밥 거의 안 해먹음
"""

In [89]:
ctx_docs = compression_retriever.get_relevant_documents(question)

In [90]:
context_text = "\n\n---\n\n".join(
    f"[CARD] {d.metadata.get('issuer')} - {d.metadata.get('card_name')}\n"
    f"[혜택]\n{d.page_content}\n"
    f"[출처] {d.metadata.get('source_url','')}\n"
    for d in ctx_docs
)

In [100]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role":"system","content":prompt},
        {"role":"system","content":f"아래는 고객 조건에 맞춰 검색된 카드 혜택 정보다. 반드시 여기서만 근거를 사용해라.\n\n{context_text}"},
        {"role":"user","content":question}
    ],
    temperature=0.8
)
print(completion.choices[0].message.content)

고객님의 프로필을 고려하여 딱 맞는 체크카드 3가지를 추천해드리겠습니다.

1. **KB국민 티머니노리학생증 체크카드(그린_세로형)**
   - 혜택: 이동통신요금 2,500원 환급할인, 스타벅스 20% 환급할인, GS25 5% 환급할인

2. **KB국민 티머니노리학생증 체크카드(블루_세로형)**
   - 혜택: 이동통신요금 2,500원 환급할인, 스타벅스 20% 환급할인, GS25 5% 환급할인

3. **KB국민 티머니노리학생증 체크카드(세로형_퍼플)**
   - 혜택: 이동통신요금 2,500원 환급할인, 스타벅스 20% 환급할인, GS25 5% 환급할인

이 세 가지 카드는 모두 이동통신요금 할인 혜택과 스타벅스, GS25 할인 혜택이 있어 고객님의 일상 소비에 도움이 될 것입니다. 또한, 외식을 자주 하시는 특이사항과 취미활동에 맞춰 식사나 카페에서의 소비를 할 때 추가 혜택을 받을 수 있습니다. 고객님의 특이사항을 고려하여 이 세 가지 카드 중에서 선택하시면 좋을 것 같습니다.


## 11. 비교, 성능평가
- base gpt-3.5-turbo 모델의 응답과 프롬프팅 및 RAG 구성후의 응답을 비교
- 추천 받은 카드를 홈페이지에서 검색하여 맞게 추천했는지 사람이 직접 검증(Human Based Evaluation)

In [101]:
prompt1 = """ 
너는 고객들에게 딱 맞는 체크카드를 추천해주는 경력 20년차 은행원이야.
고객의 프로필을 보고 딱 맞는 체크카드 3개를 추천해주고 추천하는 이유를 알려주는 게 너의 일이야.
"""
question1 = """
25세 여성 / 서울 서초구 거주(원룸 월세) / 1인 가구, 햄스터 1마리 / 스타트업 마케팅팀 사원(입사 1년 차), 블로거 
/ 아침 거름, 점심은 회사 근처 식당, 저녁은 배달·간편식·외식 위주 / 세후 월 소득 230만 원 / 예금 2000만 원, 적금 월 30만 원 / 자동차 없음(대중교통 이용) 
/ 영양제 자주 구매, 운동 자주 하지만 술 많이 마심 / 취미: 브이로그 촬영, 전시회·팝업스토어·카페 탐방, 프리다이빙, 스쿠버다이빙 / 
지출: 절반 이상 취미활동, 나머지는 월세·햄스터 용품 / 특이사항: 외식 협찬 블로그 운영, 집에서 밥 거의 안 해먹음
"""
ctx_docs = compression_retriever.get_relevant_documents(question1)
context_text = "\n\n---\n\n".join(
    f"[CARD] {d.metadata.get('issuer')} - {d.metadata.get('card_name')}\n"
    f"[혜택]\n{d.page_content}\n"
    f"[출처] {d.metadata.get('source_url','')}\n"
    for d in ctx_docs
)
completion1 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role":"system","content":prompt1},
        {"role":"system","content":f"아래는 고객 조건에 맞춰 검색된 카드 혜택 정보다. 반드시 여기서만 근거를 사용해라.\n\n{context_text}"},
        {"role":"user","content":question1}
    ],
    temperature=0
)
print(completion1.choices[0].message.content)

고객님의 프로필을 고려하여 적합한 체크카드를 3가지 추천해드리겠습니다.

1. **KB국민 티머니노리학생증 체크카드(그린_세로형)**
   - **이유**: 이 카드는 이동통신요금 2,500원 환급할인, 스타벅스 20% 환급할인, GS25 5% 환급할인 혜택이 있어서 외식 및 카페 이용에 많은 혜택을 받을 수 있습니다. 또한, 대중교통을 이용하시는 고객님께서 이동통신요금 환급 혜택을 적극 활용할 수 있을 것입니다.

2. **KB국민 티머니노리학생증 체크카드(블루_세로형)**
   - **이유**: 위와 동일한 혜택을 제공하며, 외식 및 카페 이용에 많은 혜택을 받을 수 있습니다. 또한, 이 카드는 젊은 세대를 대상으로 하는 디자인과 혜택을 제공하여 고객님의 취향에 맞을 것입니다.

3. **KB국민 티머니노리학생증 체크카드(세로형_퍼플)**
   - **이유**: 위와 동일한 혜택을 제공하며, 외식 및 카페 이용에 많은 혜택을 받을 수 있습니다. 또한, 이 카드는 세로형_퍼플로 고객님의 취향에 맞는 디자인을 제공하여 사용 시 만족도를 높일 수 있을 것입니다.

고객님의 취향과 생활 방식을 고려하여 위 3가지 카드를 추천해드렸습니다. 각 카드의 혜택을 살펴보시고, 가장 맞는 카드를 선택하시면 좋을 것 같습니다.


In [104]:
prompt3 = """ 
너는 고객들에게 딱 맞는 체크카드를 추천해주는 경력 20년차 은행원이야.
고객의 프로필을 보고 딱 맞는 체크카드 3개를 추천해주고 추천하는 이유를 알려주는 게 너의 일이야.
"""
question3 = """
25세 여성 / 서울 서초구 거주(원룸 월세) / 1인 가구, 햄스터 1마리 / 스타트업 마케팅팀 사원(입사 1년 차), 블로거 
/ 아침 거름, 점심은 회사 근처 식당, 저녁은 배달·간편식·외식 위주 / 세후 월 소득 230만 원 / 예금 2000만 원, 적금 월 30만 원 / 자동차 없음(대중교통 이용) 
/ 영양제 자주 구매, 운동 자주 하지만 술 많이 마심 / 취미: 브이로그 촬영, 전시회·팝업스토어·카페 탐방, 프리다이빙, 스쿠버다이빙 / 
지출: 절반 이상 취미활동, 나머지는 월세·햄스터 용품 / 특이사항: 외식 협찬 블로그 운영, 집에서 밥 거의 안 해먹음
"""
ctx_docs = compression_retriever.get_relevant_documents(question3)
context_text = "\n\n---\n\n".join(
    f"[CARD] {d.metadata.get('issuer')} - {d.metadata.get('card_name')}\n"
    f"[혜택]\n{d.page_content}\n"
    f"[출처] {d.metadata.get('source_url','')}\n"
    for d in ctx_docs
)
completion3 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role":"system","content":prompt3},
        {"role":"system","content":f"아래는 고객 조건에 맞춰 검색된 카드 혜택 정보다. 반드시 여기서만 근거를 사용해라.\n\n{context_text}"},
        {"role":"user","content":question3}
    ],
    temperature=0
)
print(completion3.choices[0].message.content)

주어진 고객의 프로필을 고려하여 적합한 체크카드를 추천해드리겠습니다.

1. **KB국민 티머니노리학생증 체크카드(그린_세로형)**
   - **이유**: 해당 체크카드는 이동통신요금 2,500원 환급할인, 스타벅스 20% 환급할인, GS25 5% 환급할인 혜택을 제공합니다. 고객이 외식을 자주 하고 스타벅스를 이용하는 것으로 보이므로 스타벅스 할인 혜택 및 GS25 할인 혜택이 유용할 것입니다. 또한, 이동통신요금 환급도 추가 혜택으로 받을 수 있습니다.

2. **KB국민 티머니노리학생증 체크카드(블루_세로형)**
   - **이유**: 위와 동일한 혜택을 제공하는 카드이지만, 색상이 다르므로 고객이 선호하는 색상을 선택할 수 있습니다. 블로그 운영을 하고 계신 고객께서는 외출 시 카드를 자주 사용하실 것으로 판단되므로 색상도 고려해보실 수 있습니다.

3. **KB국민 티머니노리학생증 체크카드(세로형_퍼플)**
   - **이유**: 위와 동일한 혜택을 제공하는 카드이며, 퍼플 색상으로 다양한 색상 중 하나를 선택할 수 있습니다. 블로그 활동 및 외식을 즐기시는 고객에게 어울릴 것입니다.

고객님의 프로필을 고려하여 위 3가지 체크카드를 추천해드렸습니다. 외식 및 카페 방문을 즐기시는 고객님께서는 해당 혜택을 활용하여 소비를 더욱 효율적으로 관리하실 수 있을 것입니다.


In [103]:
prompt2 = """ 
너는 고객들에게 딱 맞는 체크카드를 추천해주는 경력 20년차 은행원이야.
고객의 프로필을 보고 딱 맞는 체크카드 3개를 추천해주고 추천하는 이유를 알려주는 게 너의 일이야.
"""
question2 = """
25세 여성 / 서울 서초구 거주(원룸 월세) / 1인 가구, 햄스터 1마리 / 스타트업 마케팅팀 사원(입사 1년 차), 블로거 
/ 아침 거름, 점심은 회사 근처 식당, 저녁은 배달·간편식·외식 위주 / 세후 월 소득 230만 원 / 예금 2000만 원, 적금 월 30만 원 / 자동차 없음(대중교통 이용) 
/ 영양제 자주 구매, 운동 자주 하지만 술 많이 마심 / 취미: 브이로그 촬영, 전시회·팝업스토어·카페 탐방, 프리다이빙, 스쿠버다이빙 / 
지출: 절반 이상 취미활동, 나머지는 월세·햄스터 용품 / 특이사항: 외식 협찬 블로그 운영, 집에서 밥 거의 안 해먹음
"""
ctx_docs = compression_retriever.get_relevant_documents(question2)
context_text = "\n\n---\n\n".join(
    f"[CARD] {d.metadata.get('issuer')} - {d.metadata.get('card_name')}\n"
    f"[혜택]\n{d.page_content}\n"
    f"[출처] {d.metadata.get('source_url','')}\n"
    for d in ctx_docs
)
completion2 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role":"system","content":prompt2},
        {"role":"system","content":f"아래는 고객 조건에 맞춰 검색된 카드 혜택 정보다. 반드시 여기서만 근거를 사용해라.\n\n{context_text}"},
        {"role":"user","content":question2}
    ],
    temperature=0.8
)
print(completion2.choices[0].message.content)

주어진 고객의 프로필을 고려하여 적합한 체크카드 3가지를 추천해 드리겠습니다.

1. **KB국민 티머니노리학생증 체크카드(그린_세로형)**
   - **이유**: 이 카드는 이동통신요금 2,500원 환급할인, 스타벅스 20% 환급할인, GS25 5% 환급할인 혜택이 있어서 외식 및 스타벅스 이용에 적합합니다. 또한, 주로 외식을 하고 취미활동에 지출하는 고객님에게 적합한 혜택이 제공됩니다.

2. **KB국민 티머니노리학생증 체크카드(블루_세로형)**
   - **이유**: 위와 동일한 KB국민카드의 체크카드로, 이동통신요금 2,500원 환급할인, 스타벅스 20% 환급할인, GS25 5% 환급할인 혜택을 제공합니다. 외식 및 스타벅스 이용시 혜택을 받을 수 있어서 고객님의 소비 패턴에 적합합니다.

3. **KB국민 티머니노리학생증 체크카드(세로형_퍼플)**
   - **이유**: 마지막으로도 KB국민카드의 이 카드는 이동통신요금 2,500원 환급할인, 스타벅스 20% 환급할인, GS25 5% 환급할인 혜택을 가지고 있습니다. 많은 외식 및 스타벅스 이용에 따른 혜택을 받을 수 있으므로 고객님의 소비 습관과 일상에 부합하는 카드 중 하나입니다.

고객님의 프로필을 고려하여 위 3가지 카드를 추천해 드렸습니다. 적합한 카드를 선택하여 일상 소비 및 취미활동에 더 많은 혜택을 누리시길 바랍니다.


In [126]:
prompt4 = """ 
너는 고객들에게 딱 맞는 체크카드를 추천해주는 경력 20년차 은행원이야.
고객의 프로필을 보고 딱 맞는 체크카드 3개를 추천해주고 추천하는 이유를 알려주는 게 너의 일이야.
"""
question4 = """
25세 여성 / 서울 서초구 거주(원룸 월세) / 1인 가구, 햄스터 1마리 / 스타트업 마케팅팀 사원(입사 1년 차), 블로거 
/ 아침 거름, 점심은 회사 근처 식당, 저녁은 배달·간편식·외식 위주 / 세후 월 소득 230만 원 / 예금 2000만 원, 적금 월 30만 원 / 자동차 없음(대중교통 이용) 
/ 영양제 자주 구매, 운동 자주 하지만 술 많이 마심 / 취미: 브이로그 촬영, 전시회·팝업스토어·카페 탐방, 프리다이빙, 스쿠버다이빙 / 
지출: 절반 이상 취미활동, 나머지는 월세·햄스터 용품 / 특이사항: 외식 협찬 블로그 운영, 집에서 밥 거의 안 해먹음
"""
ctx_docs = compression_retriever.get_relevant_documents(question4)
context_text = "\n\n---\n\n".join(
    f"[CARD] {d.metadata.get('issuer')} - {d.metadata.get('card_name')}\n"
    f"[혜택]\n{d.page_content}\n"
    f"[출처] {d.metadata.get('source_url','')}\n"
    for d in ctx_docs
)
completion4 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role":"system","content":prompt4},
        {"role":"system","content":f"아래는 고객 조건에 맞춰 검색된 카드 혜택 정보다. 반드시 여기서만 근거를 사용해라.\n\n{context_text}"},
        {"role":"user","content":question4}
    ],
    temperature=0.8
)
print(completion4.choices[0].message.content)

주어진 고객 프로필을 고려해보면, KB국민카드의 티머니노리학생증 체크카드 중에서 세로형_퍼플 체크카드를 추천해드리고 싶어요. 이 카드는 이동통신요금 2,500원 환급할인, 스타벅스 20% 환급할인, GS25 5% 환급할인 혜택을 제공합니다.

고객님은 절반 이상을 취미활동에 지출하시는 것으로 보입니다. 블로거이자 스쿠버다이빙, 프리다이빙을 즐기는 분으로 보아 해외여행이나 액티비티를 즐기실 가능성이 높은데, 이때 스타벅스나 편의점에서의 할인 혜택이 유용할 것입니다. 또한, 점심은 회사 근처 식당에서 섭취하시고 저녁은 배달이나 외식 위주로 하시는데, GS25의 환급 혜택 또한 소비 패턴에 잘 맞을 것입니다.

또한, 금융 활동이 활발하시기 때문에 소득이 낮아도 혜택을 최대한 누릴 수 있는 카드를 선택하는 것이 중요합니다. 세후 월 소득 230만 원에 비해 예금과 적금을 통한 저축이 활발해 보입니다. 따라서, 이러한 혜택을 통해 소비적립을 더 활발하게 할 수 있을 것입니다.

그러므로, 고객님의 프로필을 고려할 때 KB국민카드의 티머니노리학생증 체크카드(세로형_퍼플)를 추천해드리고, 이유를 설명드렸습니다. 혹시 추가로 궁금한 점이 있으면 물어봐 주세요.


In [108]:
prompt5 = """ 
너는 고객들에게 딱 맞는 체크카드를 추천해주는 경력 20년차 은행원이야.
고객의 프로필을 보고 딱 맞는 체크카드 3개를 추천해주고 추천하는 이유를 알려주는 게 너의 일이야.
"""
question5 = """
25세 여성 / 서울 서초구 거주(원룸 월세) / 1인 가구, 햄스터 1마리 / 스타트업 마케팅팀 사원(입사 1년 차), 블로거 
/ 아침 거름, 점심은 회사 근처 식당, 저녁은 배달·간편식·외식 위주 / 세후 월 소득 230만 원 / 예금 2000만 원, 적금 월 30만 원 / 자동차 없음(대중교통 이용) 
/ 영양제 자주 구매, 운동 자주 하지만 술 많이 마심 / 취미: 브이로그 촬영, 전시회·팝업스토어·카페 탐방, 프리다이빙, 스쿠버다이빙 / 
지출: 절반 이상 취미활동, 나머지는 월세·햄스터 용품 / 특이사항: 외식 협찬 블로그 운영, 집에서 밥 거의 안 해먹음
"""
ctx_docs = compression_retriever.get_relevant_documents(question5)
context_text = "\n\n---\n\n".join(
    f"[CARD] {d.metadata.get('issuer')} - {d.metadata.get('card_name')}\n"
    f"[혜택]\n{d.page_content}\n"
    f"[출처] {d.metadata.get('source_url','')}\n"
    for d in ctx_docs
)
completion5 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role":"system","content":prompt5},
        {"role":"system","content":f"아래는 고객 조건에 맞춰 검색된 카드 혜택 정보다. 반드시 여기서만 근거를 사용해라.\n\n{context_text}"},
        {"role":"user","content":question5}
    ],
    temperature=0.5
)
print(completion5.choices[0].message.content)

주어진 고객 프로필을 고려하여 적합한 체크카드 3가지를 추천해드리겠습니다.

1. **KB국민 티머니노리학생증 체크카드(그린_세로형)**
   - 혜택: 이동통신요금 2,500원 환급할인, 스타벅스 20% 환급할인, GS25 5% 환급할인
   - 추천 이유: 이동통신 요금 할인 혜택과 스타벅스, GS25 할인 혜택이 있어서 휴대폰 이용 및 외식에 많이 소비하는 고객님에게 적합할 것으로 보입니다.

2. **KB국민 티머니노리학생증 체크카드(블루_세로형)**
   - 혜택: 이동통신요금 2,500원 환급할인, 스타벅스 20% 환급할인, GS25 5% 환급할인
   - 추천 이유: 위와 동일한 혜택으로, 휴대폰 이용 및 외식에 많이 소비하는 고객님에게 부가 혜택을 제공할 수 있을 것입니다.

3. **KB국민 티머니노리학생증 체크카드(세로형_퍼플)**
   - 혜택: 이동통신요금 2,500원 환급할인, 스타벅스 20% 환급할인, GS25 5% 환급할인
   - 추천 이유: 위와 동일한 혜택을 제공하며, 이동통신 요금 할인 혜택이 있어서 휴대폰 이용에 많이 소비하는 고객님에게 적합합니다.

고객님의 프로필을 고려하여 위 3가지 체크카드를 추천해드렸습니다. 특히 이동통신 요금 할인 및 외식 관련 혜택이 많은 카드가 선택되었습니다. 부디 이를 참고하여 적합한 카드를 선택해주시기 바랍니다.


In [109]:
prompt6 = """ 
너는 고객들에게 딱 맞는 체크카드를 추천해주는 경력 20년차 은행원이야.
고객의 프로필을 보고 딱 맞는 체크카드 3개를 추천해주고 추천하는 이유를 알려주는 게 너의 일이야.
"""
question6 = """
25세 여성 / 서울 서초구 거주(원룸 월세) / 1인 가구, 햄스터 1마리 / 스타트업 마케팅팀 사원(입사 1년 차), 블로거 
/ 아침 거름, 점심은 회사 근처 식당, 저녁은 배달·간편식·외식 위주 / 세후 월 소득 230만 원 / 예금 2000만 원, 적금 월 30만 원 / 자동차 없음(대중교통 이용) 
/ 영양제 자주 구매, 운동 자주 하지만 술 많이 마심 / 취미: 브이로그 촬영, 전시회·팝업스토어·카페 탐방, 프리다이빙, 스쿠버다이빙 / 
지출: 절반 이상 취미활동, 나머지는 월세·햄스터 용품 / 특이사항: 외식 협찬 블로그 운영, 집에서 밥 거의 안 해먹음
"""
ctx_docs = compression_retriever.get_relevant_documents(question6)
context_text = "\n\n---\n\n".join(
    f"[CARD] {d.metadata.get('issuer')} - {d.metadata.get('card_name')}\n"
    f"[혜택]\n{d.page_content}\n"
    f"[출처] {d.metadata.get('source_url','')}\n"
    for d in ctx_docs
)
completion6 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role":"system","content":prompt6},
        {"role":"system","content":f"아래는 고객 조건에 맞춰 검색된 카드 혜택 정보다. 반드시 여기서만 근거를 사용해라.\n\n{context_text}"},
        {"role":"user","content":question6}
    ],
    temperature=0.5
)
print(completion6.choices[0].message.content)

주어진 고객의 프로필을 고려하여 적합한 체크카드 3가지를 추천해드리겠습니다.

1. **KB국민 티머니노리학생증 체크카드(그린_세로형)**
   - 혜택: 이동통신요금 2,500원 환급할인, 스타벅스 20% 환급할인, GS25 5% 환급할인
   - 이유: 이동통신 요금 환급, 스타벅스 및 편의점 할인 혜택이 있어서 외식 및 편의점 이용에 많이 도움이 될 것입니다.

2. **KB국민 티머니노리학생증 체크카드(블루_세로형)**
   - 혜택: 이동통신요금 2,500원 환급할인, 스타벅스 20% 환급할인, GS25 5% 환급할인
   - 이유: 이동통신 요금 환급, 스타벅스 및 편의점 할인 혜택이 있어서 외식 및 편의점 이용에 많이 도움이 될 것입니다.

3. **KB국민 티머니노리학생증 체크카드(세로형_퍼플)**
   - 혜택: 이동통신요금 2,500원 환급할인, 스타벅스 20% 환급할인, GS25 5% 환급할인
   - 이유: 이동통신 요금 환급, 스타벅스 및 편의점 할인 혜택이 있어서 외식 및 편의점 이용에 많이 도움이 될 것입니다.

고객님의 프로필을 고려하여 이동통신 요금 할인 및 외식, 편의점 할인 혜택이 있는 카드를 추천해드렸습니다. 외식 및 편의점 이용이 많으시기 때문에 이러한 혜택이 가장 맞을 것으로 판단됩니다.


In [110]:
prompt7 = """ 
너는 고객들에게 딱 맞는 체크카드를 추천해주는 경력 20년차 은행원이야.
고객의 프로필을 보고 딱 맞는 체크카드 3개를 추천해주고 추천하는 이유를 알려주는 게 너의 일이야.
"""
question7 = """
25세 여성 / 서울 서초구 거주(원룸 월세) / 1인 가구, 햄스터 1마리 / 스타트업 마케팅팀 사원(입사 1년 차), 블로거 
/ 아침 거름, 점심은 회사 근처 식당, 저녁은 배달·간편식·외식 위주 / 세후 월 소득 230만 원 / 예금 2000만 원, 적금 월 30만 원 / 자동차 없음(대중교통 이용) 
/ 영양제 자주 구매, 운동 자주 하지만 술 많이 마심 / 취미: 브이로그 촬영, 전시회·팝업스토어·카페 탐방, 프리다이빙, 스쿠버다이빙 / 
지출: 절반 이상 취미활동, 나머지는 월세·햄스터 용품 / 특이사항: 외식 협찬 블로그 운영, 집에서 밥 거의 안 해먹음
"""
ctx_docs = compression_retriever.get_relevant_documents(question7)
context_text = "\n\n---\n\n".join(
    f"[CARD] {d.metadata.get('issuer')} - {d.metadata.get('card_name')}\n"
    f"[혜택]\n{d.page_content}\n"
    f"[출처] {d.metadata.get('source_url','')}\n"
    for d in ctx_docs
)
completion7 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role":"system","content":prompt7},
        {"role":"system","content":f"아래는 고객 조건에 맞춰 검색된 카드 혜택 정보다. 반드시 여기서만 근거를 사용해라.\n\n{context_text}"},
        {"role":"user","content":question7}
    ],
    temperature=1
)
print(completion7.choices[0].message.content)

주어진 정보를 바탕으로 여성 고객에게 추천하는 체크카드 세 가지는 다음과 같습니다:

1. **KB국민 티머니노리학생증 체크카드(그린_세로형)**
   - **이유**: 이 카드는 이동통신요금 2,500원 환급 및 스타벅스 20% 할인, GS25 5% 할인 혜택을 제공하여 외식 및 여가 활동에 많은 혜택을 줍니다. 
   
2. **KB국민 티머니노리학생증 체크카드(블루_세로형)**
   - **이유**: 마찬가지로, 서울 서초구 거주에 적합한 카드로 이동통신요금 2,500원 환급 및 스타벅스 20% 할인, GS25 5% 할인 혜택을 제공합니다. 외식 및 편의점 이용이 빈번한 고객에게 유용할 것입니다.
   
3. **KB국민 티머니노리학생증 체크카드(세로형_퍼플)**
   - **이유**: 영양제 구매 및 운동을 즐기는 고객에게 유리한 혜택을 제공합니다. 이동통신요금 2,500원 환급 및 스타벅스 20% 할인, GS25 5% 할인 혜택이 있습니다. 술을 자주 마시지만 외식을 즐기는 고객에게 권해드립니다.

고객의 프로필과 취향을 고려하여 여러 혜택 중 외식 및 여가 활동, 그리고 영양제 및 운동 관련 혜택을 중심으로 추천해드렸습니다. 외식과 레저를 즐기시는 고객에게 특히 맞는 카드들이니 참고 부탁드립니다.


In [111]:
prompt8 = """ 
너는 고객들에게 딱 맞는 체크카드를 추천해주는 경력 20년차 은행원이야.
고객의 프로필을 보고 딱 맞는 체크카드 3개를 추천해주고 추천하는 이유를 알려주는 게 너의 일이야.
"""
question8 = """
25세 여성 / 서울 서초구 거주(원룸 월세) / 1인 가구, 햄스터 1마리 / 스타트업 마케팅팀 사원(입사 1년 차), 블로거 
/ 아침 거름, 점심은 회사 근처 식당, 저녁은 배달·간편식·외식 위주 / 세후 월 소득 230만 원 / 예금 2000만 원, 적금 월 30만 원 / 자동차 없음(대중교통 이용) 
/ 영양제 자주 구매, 운동 자주 하지만 술 많이 마심 / 취미: 브이로그 촬영, 전시회·팝업스토어·카페 탐방, 프리다이빙, 스쿠버다이빙 / 
지출: 절반 이상 취미활동, 나머지는 월세·햄스터 용품 / 특이사항: 외식 협찬 블로그 운영, 집에서 밥 거의 안 해먹음
"""
ctx_docs = compression_retriever.get_relevant_documents(question8)
context_text = "\n\n---\n\n".join(
    f"[CARD] {d.metadata.get('issuer')} - {d.metadata.get('card_name')}\n"
    f"[혜택]\n{d.page_content}\n"
    f"[출처] {d.metadata.get('source_url','')}\n"
    for d in ctx_docs
)
completion8 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role":"system","content":prompt8},
        {"role":"system","content":f"아래는 고객 조건에 맞춰 검색된 카드 혜택 정보다. 반드시 여기서만 근거를 사용해라.\n\n{context_text}"},
        {"role":"user","content":question8}
    ],
    temperature=1
)
print(completion8.choices[0].message.content)

안녕하세요. 고객님의 프로필을 고려하여 적합한 체크카드를 3가지 추천해드리겠습니다.

1. **KB국민 티머니노리학생증 체크카드(그린_세로형)**
   - 혜택: 이동통신요금 2,500원 환급할인, 스타벅스 20% 환급할인, GS25 5% 환급할인
   - 해당 체크카드는 외식 및 카페 이용을 많이 하시는 고객님께 적합합니다. 스타벅스 및 GS25 할인 혜택을 통해 지출을 조금 더 절약하실 수 있습니다.

2. **KB국민 티머니노리학생증 체크카드(블루_세로형)**
   - 혜택: 이동통신요금 2,500원 환급할인, 스타벅스 20% 환급할인, GS25 5% 환급할인
   - 블로거로서 외식 협찬 활동을 많이 하시는 고객님께서는 이 체크카드를 통해 외식 비용을 조금 더 절약하실 수 있습니다. 

3. **KB국민 티머니노리학생증 체크카드(세로형_퍼플)**
   - 혜택: 이동통신요금 2,500원 환급할인, 스타벅스 20% 환급할인, GS25 5% 환급할인
   - 블로그 운영 및 자주 찍는 브이로그 촬영 등 다양한 활동을 좋아하시는 고객님께 적합한 체크카드입니다. 스타벅스 할인 혜택을 통해 카페에서 영감을 얻는 시간을 더 즐기실 수 있습니다.

고객님의 취미와 지출 형태를 고려하여 위 3가지의 체크카드를 추천해드렸습니다. 어느 체크카드를 선택하셔도 혜택을 잘 활용하실 수 있을 것으로 예상됩니다. 


In [112]:
prompt9 = """ 
너는 고객들에게 딱 맞는 체크카드를 추천해주는 경력 20년차 은행원이야.
고객의 프로필을 보고 딱 맞는 체크카드 3개를 추천해주고 추천하는 이유를 알려주는 게 너의 일이야.
"""
question9 = """
25세 여성 / 서울 서초구 거주(원룸 월세) / 1인 가구, 햄스터 1마리 / 스타트업 마케팅팀 사원(입사 1년 차), 블로거 
/ 아침 거름, 점심은 회사 근처 식당, 저녁은 배달·간편식·외식 위주 / 세후 월 소득 230만 원 / 예금 2000만 원, 적금 월 30만 원 / 자동차 없음(대중교통 이용) 
/ 영양제 자주 구매, 운동 자주 하지만 술 많이 마심 / 취미: 브이로그 촬영, 전시회·팝업스토어·카페 탐방, 프리다이빙, 스쿠버다이빙 / 
지출: 절반 이상 취미활동, 나머지는 월세·햄스터 용품 / 특이사항: 외식 협찬 블로그 운영, 집에서 밥 거의 안 해먹음
"""
ctx_docs = compression_retriever.get_relevant_documents(question9)
context_text = "\n\n---\n\n".join(
    f"[CARD] {d.metadata.get('issuer')} - {d.metadata.get('card_name')}\n"
    f"[혜택]\n{d.page_content}\n"
    f"[출처] {d.metadata.get('source_url','')}\n"
    for d in ctx_docs
)
completion9 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role":"system","content":prompt9},
        {"role":"system","content":f"아래는 고객 조건에 맞춰 검색된 카드 혜택 정보다. 반드시 여기서만 근거를 사용해라.\n\n{context_text}"},
        {"role":"user","content":question9}
    ],
    temperature=1
)
print(completion9.choices[0].message.content)

고객님의 프로필을 고려하여 적합한 체크카드를 3가지 추천해 드리겠습니다.

1. **KB국민 티머니노리학생증 체크카드(그린_세로형)**
   - 혜택: 이동통신요금 2,500원 환급할인, 스타벅스 20% 환급할인, GS25 5% 환급할인
   - 이유: 월세 등 생활비를 지출하는 고객님에게 이동통신요금 환급과 스타벅스, GS25 할인 혜택이 유용할 것으로 판단됩니다. 또한 외식 및 간편식 위주의 지출에도 도움이 될 수 있습니다.

2. **KB국민 티머니노리학생증 체크카드(블루_세로형)**
   - 혜택: 이동통신요금 2,500원 환급할인, 스타벅스 20% 환급할인, GS25 5% 환급할인
   - 이유: 혜택이 그린_세로형과 동일하며, 스타트업 마케팅팀 사원이신 고객님에게 이동통신요금 환급 혜택은 특히 유용할 것으로 판단됩니다.

3. **KB국민 티머니노리학생증 체크카드(세로형_퍼플)**
   - 혜택: 이동통신요금 2,500원 환급할인, 스타벅스 20% 환급할인, GS25 5% 환급할인
   - 이유: 위의 두 가지 카드와 마찬가지로 유용한 혜택을 제공하며, 색상과 디자인까지 고객님의 취향과 성향에 맞을 수 있습니다. 브이로그 촬영 등 취미 활동 시 사용하기 좋을 것입니다.

고객님의 생활 방식 및 소비 성향을 고려하여 위 세 가지 체크카드를 추천해 드렸습니다. 필요하신 경우 해당 카드의 자세한 내용을 은행에서 확인해 보시기 바랍니다.


- 베이스

In [114]:
llm = ChatOpenAI(model = 'gpt-3.5-turbo',
                 temperature = 0.8,
                 api_key = MY_API_KEY
                )
base_prompt = f'''
너는 카드 추천 어시스턴트야.
사용자의 프로필과 요청에 맞춰 카드 3개를 추천하고,
각 카드의 근거(혜택-조건)를 설명해 줘.
[사용자 프로필]
{question}
[요청]
이 생활 패턴에 맞는 카드를 추천해 줘.
'''
print(llm.invoke(base_prompt).content)

1. 현대카드 The CLASSIC
- 혜택: 외식 및 카페 이용 시 할인 및 적립 혜택
- 조건: 월 소득 200만 원 이상
- 이유: 외식 및 카페를 자주 이용하고, 취미로 블로그를 운영하며 외식 협찬을 받는 사용자에게 적합한 카드입니다.

2. 삼성카드 S20
- 혜택: 영화, 전시회, 팝업스토어 이용 시 할인 및 적립 혜택
- 조건: 신용등급 A 이상
- 이유: 전시회, 팝업스토어 등을 자주 탐방하고, 취미활동에 지출하는 사용자에게 적합한 카드입니다.

3. KB국민카드 The LIVE
- 혜택: 온라인 쇼핑 및 운동 용품 구매 시 할인 및 적립 혜택
- 조건: 월 소득 150만 원 이상
- 이유: 영양제나 운동 용품을 자주 구매하고, 술을 많이 마시는 사용자에게 적합한 카드입니다.


In [115]:
llm = ChatOpenAI(model = 'gpt-3.5-turbo',
                 temperature = 0.8,
                 api_key = MY_API_KEY
                )
base_prompt = f'''
너는 카드 추천 어시스턴트야.
사용자의 프로필과 요청에 맞춰 카드 3개를 추천하고,
각 카드의 근거(혜택-조건)를 설명해 줘.
[사용자 프로필]
{question}
[요청]
이 생활 패턴에 맞는 카드를 추천해 줘.
'''
print(llm.invoke(base_prompt).content)

고객님에게 적합한 카드 추천을 위해 고려한 사항은 다음과 같습니다.

1. 식비 및 외식 관련 혜택: 고객님은 저녁을 배달이나 간편식, 외식 위주로 선호하시는 것으로 보입니다. 따라서 식비 할인 혜택이 있는 카드를 추천해 드릴 수 있습니다.

2. 취미 관련 혜택: 고객님은 브이로그 촬영, 전시회·팝업스토어·카페 탐방, 프리다이빙, 스쿠버다이빙 등 다양한 취미 활동을 즐기시는 것으로 보입니다. 해당 취미 관련 혜택이 있는 카드를 추천해 드릴 수 있습니다.

3. 온라인 쇼핑 혜택: 운동을 자주 하시고 영양제를 자주 구매하시는 것으로 보입니다. 온라인 쇼핑 관련 혜택이 있는 카드를 추천해 드릴 수 있습니다.

위의 고려사항을 바탕으로 고객님에게 추천드리는 카드는 다음과 같습니다.

1. 삼성카드
- 혜택: 외식 및 배달 음식 결제 시 할인 혜택 제공
- 조건: 월 소득 200만 원 이상
- 기타 혜택: 온라인 쇼핑 및 여가 활동에 대한 할인 및 적립 혜택

2. 현대카드
- 혜택: 다이빙 및 스포츠 관련 이벤트 및 할인 혜택 제공
- 조건: 월 소득 150만 원 이상
- 기타 혜택: 식비 할인 및 취미 활동 관련 혜택 제공

3. 국민카드
- 혜택: 블로그 운영에 도움이 되는 컨텐츠 제작 및 마케팅 지원 혜택 제공
- 조건: 월 소득 180만 원 이상
- 기타 혜택: 다양한 취미 활동 및 온라인 쇼핑 혜택 제공

위의 카드 중에서 고객님의 요구사항 및 생활 패턴에 가장 적합한 카드를 선택해 보시기 바랍니다. 혹시 추가적인 질문이나 요청이 있으시면 언제든지 문의해 주세요.


In [116]:
llm = ChatOpenAI(model = 'gpt-3.5-turbo',
                 temperature = 0.8,
                 api_key = MY_API_KEY
                )
base_prompt = f'''
너는 카드 추천 어시스턴트야.
사용자의 프로필과 요청에 맞춰 카드 3개를 추천하고,
각 카드의 근거(혜택-조건)를 설명해 줘.
[사용자 프로필]
{question}
[요청]
이 생활 패턴에 맞는 카드를 추천해 줘.
'''
print(llm.invoke(base_prompt).content)

1. 현대카드 The CLASSIC
- 혜택: 외식 및 온라인 쇼핑 결제 시 최대 5% 캐시백, 공항 라운지 무료 이용
- 조건: 연 회비 10만 원, 월 20만 원 이상 사용 시 연 회비 할인 혜택

2. 삼성카드 Galaxy Card
- 혜택: 빅스비 결제 시 최대 10% 캐시백, 전국 주유소 3% 할인
- 조건: 연 회비 없음, 삼성페이 사용 시 추가 혜택

3. 씨티카드 마일리지 플러스
- 혜택: 해외쇼핑 2배 마일리지 적립, 수십만개의 제휴점에서 할인 혜택
- 조건: 연 회비 7만 원, 매월 최소 5만 원 이상 사용 시 연 회비 할인 혜택, 해외 이용 시 추가 혜택

이 세 가지 카드는 외식 및 온라인 쇼핑에 많이 사용하는 당신의 생활 패턴에 적합한 카드입니다. 더불어 각 카드의 혜택을 최대한 활용하여 소득을 늘리거나 취미활동을 더욱 즐길 수 있을 것입니다.


In [117]:
llm = ChatOpenAI(model = 'gpt-3.5-turbo',
                 temperature = 1,
                 api_key = MY_API_KEY
                )
base_prompt = f'''
너는 카드 추천 어시스턴트야.
사용자의 프로필과 요청에 맞춰 카드 3개를 추천하고,
각 카드의 근거(혜택-조건)를 설명해 줘.
[사용자 프로필]
{question}
[요청]
이 생활 패턴에 맞는 카드를 추천해 줘.
'''
print(llm.invoke(base_prompt).content)

안녕하세요! 사용자님의 생활 패턴을 고려하여 여러 카드 중에서 3가지를 추천해 드리겠습니다.

1. 하나카드 리워즈 신용카드
- 혜택: 외식 및 온라인 쇼핑 시 적립률 향상, 전월 카드 사용액에 따라 캐시백 혜택 제공
- 조건: 월 카드 사용금액이 50만 원 이상일 때 캐시백 혜택 적용

2. 삼성카드 The Select 신용카드
- 혜택: 문화 생활에 대한 특별 할인 및 적립 실적에 따른 VIP 서비스 제공
- 조건: 전시회 및 팝업스토어 방문 시 특별 할인 혜택 제공

3. 현대카드 Hi-Point 신용카드
- 혜택: 온라인 쇼핑 및 영양제 구매 시 적립률 향상, 월 결제금액에 따른 포인트 적립
- 조건: 영양제 및 운동 용품 구매 시 적립률이 높아 결제 시 주의가 필요

이 세 가지 카드는 사용자님의 생활 패턴과 취향을 고려하여 추천해 드린 것이니 참고 부탁드립니다. 각 카드의 상세한 내용은 해당 카드사의 공식 홈페이지에서 확인하실 수 있습니다. 더 궁금한 점이 있으시면 언제든지 물어봐 주세요!


In [118]:
llm = ChatOpenAI(model = 'gpt-3.5-turbo',
                 temperature = 1,
                 api_key = MY_API_KEY
                )
base_prompt = f'''
너는 카드 추천 어시스턴트야.
사용자의 프로필과 요청에 맞춰 카드 3개를 추천하고,
각 카드의 근거(혜택-조건)를 설명해 줘.
[사용자 프로필]
{question}
[요청]
이 생활 패턴에 맞는 카드를 추천해 줘.
'''
print(llm.invoke(base_prompt).content)

고객님께 추천하는 카드는 다음과 같습니다:

1. 현대카드 The Classic
- 혜택: 외식 및 온라인 쇼핑에 적합한 카드로, 매월 할인 혜택 및 적립 포인트 제공
- 조건: 연회비 면제 조건이 있으며, 월 소득이 200만 원 이상이어야 합니다.

2. 삼성카드 Green
- 혜택: 다양한 영화, 전시회, 카페 할인 혜택 및 운동 관련 혜택 제공
- 조건: 연회비가 저렴하며, 외식 및 문화 활동에 다양한 혜택을 제공

3. 국민카드 MOMENT
- 혜택: 라이프스타일 및 여가 취미에 적합한 혜택 제공
- 조건: 연회비 면제 및 다양한 이벤트 및 혜택 제공

위의 카드는 고객님의 라이프스타일 및 소비패턴을 고려하여 추천드리는 것이니, 자세한 사항은 해당 카드사 홈페이지를 참고하시기 바랍니다. 감사합니다.


In [119]:
llm = ChatOpenAI(model = 'gpt-3.5-turbo',
                 temperature = 1,
                 api_key = MY_API_KEY
                )
base_prompt = f'''
너는 카드 추천 어시스턴트야.
사용자의 프로필과 요청에 맞춰 카드 3개를 추천하고,
각 카드의 근거(혜택-조건)를 설명해 줘.
[사용자 프로필]
{question}
[요청]
이 생활 패턴에 맞는 카드를 추천해 줘.
'''
print(llm.invoke(base_prompt).content)

1. 현대카드 Zero
- 혜택: 외식 및 배달 음식 할인 및 캐시백
- 조건: 월 소득 230만 원 이하 시 사용 가능, 전 카테고리 최대 40% 할인

2. 신한카드 The Select Visa
- 혜택: 온라인 쇼핑, 전시회 입장료 할인 혜택
- 조건: 연회비 면제, 해외에서도 사용 가능, 온라인 쇼핑 최대 5% 적립

3. 국민카드 YOLO 골드
- 혜택: 레저 및 문화 활동 할인 혜택
- 조건: 대중교통 결제 시 최대 20% 할인, 예금 연동 시 추가 혜택제공

이 세 가지 카드는 사용자의 생활 패턴과 취향에 맞게 선택할 수 있을 것입니다. 저렴한 외식 할인부터 레저 및 문화 활동까지 다양한 혜택을 제공하여 일상 생활에 도움을 줄 수 있습니다. 선택 시 참고하시기 바랍니다.


In [120]:
llm = ChatOpenAI(model = 'gpt-3.5-turbo',
                 temperature = 0,
                 api_key = MY_API_KEY
                )
base_prompt = f'''
너는 카드 추천 어시스턴트야.
사용자의 프로필과 요청에 맞춰 카드 3개를 추천하고,
각 카드의 근거(혜택-조건)를 설명해 줘.
[사용자 프로필]
{question}
[요청]
이 생활 패턴에 맞는 카드를 추천해 줘.
'''
print(llm.invoke(base_prompt).content)

1. 현대카드 The CLASSIC
- 혜택: 외식 및 카페 이용 시 할인 혜택 제공
- 조건: 월 소득 200만 원 이상, 1년 이상 재직한 직장인에게 제공

2. 삼성카드 S20
- 혜택: 온라인 쇼핑 및 배달 음식 주문 시 적립 및 할인 혜택 제공
- 조건: 월 소득 200만 원 이상, 1년 이상 재직한 직장인에게 제공

3. KB국민카드 The LIVE
- 혜택: 문화생활 및 레저 활동 시 할인 혜택 제공
- 조건: 월 소득 200만 원 이상, 1년 이상 재직한 직장인에게 제공

이 세 가지 카드는 사용자의 생활 패턴과 취향에 맞는 혜택을 제공하며, 월 소득과 재직 기간 등의 조건을 충족하면 발급이 가능합니다. 외식, 온라인 쇼핑, 문화생활 등 다양한 활동을 즐기는 사용자에게 적합한 카드입니다.


In [121]:
llm = ChatOpenAI(model = 'gpt-3.5-turbo',
                 temperature = 0,
                 api_key = MY_API_KEY
                )
base_prompt = f'''
너는 카드 추천 어시스턴트야.
사용자의 프로필과 요청에 맞춰 카드 3개를 추천하고,
각 카드의 근거(혜택-조건)를 설명해 줘.
[사용자 프로필]
{question}
[요청]
이 생활 패턴에 맞는 카드를 추천해 줘.
'''
print(llm.invoke(base_prompt).content)

고객님의 생활 패턴에 적합한 카드 추천을 드리겠습니다.

1. 현대카드 The CLASSIC
- 혜택: 외식 및 온라인 쇼핑 결제 시 할인 혜택 제공
- 조건: 월 소득 200만 원 이상, 20대 이상 신용등급 6등급 이상

2. 삼성카드 S20
- 혜택: 온라인 쇼핑 및 교통 이용 시 적립 및 할인 혜택 제공
- 조건: 월 소득 200만 원 이상, 20대 이상 신용등급 6등급 이상

3. KB국민카드 The LIVE
- 혜택: 문화·영화·콘서트 등 다양한 문화생활 할인 혜택 제공
- 조건: 월 소득 200만 원 이상, 20대 이상 신용등급 6등급 이상

고객님의 취향과 생활 패턴을 고려하여 위 3가지 카드를 추천해 드렸습니다. 각 카드의 혜택과 조건을 참고하시어 적합한 카드를 선택해 보시기 바랍니다. 감사합니다.


In [122]:
llm = ChatOpenAI(model = 'gpt-3.5-turbo',
                 temperature = 0,
                 api_key = MY_API_KEY
                )
base_prompt = f'''
너는 카드 추천 어시스턴트야.
사용자의 프로필과 요청에 맞춰 카드 3개를 추천하고,
각 카드의 근거(혜택-조건)를 설명해 줘.
[사용자 프로필]
{question}
[요청]
이 생활 패턴에 맞는 카드를 추천해 줘.
'''
print(llm.invoke(base_prompt).content)

고객님의 생활 패턴에 적합한 카드 추천을 드리겠습니다.

1. 현대카드 The CLASSIC
- 혜택: 외식 및 온라인 쇼핑 결제 시 할인 혜택 제공
- 조건: 월 소득 200만 원 이상, 20대 이상 신용등급 6등급 이상

2. 삼성카드 S20
- 혜택: 온라인 쇼핑 및 교통카드 결제 시 적립 및 할인 혜택 제공
- 조건: 월 소득 200만 원 이상, 20대 이상 신용등급 6등급 이상

3. KB국민카드 The LIVE
- 혜택: 문화·영화·콘서트 등 다양한 문화생활 할인 혜택 제공
- 조건: 월 소득 200만 원 이상, 20대 이상 신용등급 6등급 이상

고객님의 취향과 생활 패턴을 고려하여 위 3가지 카드를 추천해 드렸습니다. 각 카드의 혜택과 조건을 참고하시어 적합한 카드를 선택해 보시기 바랍니다. 감사합니다.


In [123]:
llm = ChatOpenAI(model = 'gpt-3.5-turbo',
                 temperature = 0.5,
                 api_key = MY_API_KEY
                )
base_prompt = f'''
너는 카드 추천 어시스턴트야.
사용자의 프로필과 요청에 맞춰 카드 3개를 추천하고,
각 카드의 근거(혜택-조건)를 설명해 줘.
[사용자 프로필]
{question}
[요청]
이 생활 패턴에 맞는 카드를 추천해 줘.
'''
print(llm.invoke(base_prompt).content)

1. 현대카드 The Classic
- 혜택: 외식 및 온라인 쇼핑 할인 혜택 제공
- 조건: 월 10만 원 이상 이용 시 5% 할인, 온라인 쇼핑 시 3% 할인

2. 삼성카드 T멤버십
- 혜택: 쇼핑 및 엔터테인먼트 할인 혜택 제공
- 조건: T멤버십 회원으로 가입 시 매달 다양한 할인 혜택 제공

3. KB국민카드 The Select
- 혜택: 영화 및 문화생활 할인 혜택 제공
- 조건: 영화 및 전시회 관람 시 할인 혜택 제공, 카페 및 레스토랑 할인 혜택도 제공

이 세 가지 카드는 사용자의 취향과 생활 패턴에 맞춰 다양한 혜택을 제공하며, 외식 및 쇼핑 등 다양한 활동에 유용할 것입니다. 혜택을 최대한 활용하여 일상 소비를 절약하고 즐거운 삶을 누리시기 바랍니다.


In [124]:
llm = ChatOpenAI(model = 'gpt-3.5-turbo',
                 temperature = 0.5,
                 api_key = MY_API_KEY
                )
base_prompt = f'''
너는 카드 추천 어시스턴트야.
사용자의 프로필과 요청에 맞춰 카드 3개를 추천하고,
각 카드의 근거(혜택-조건)를 설명해 줘.
[사용자 프로필]
{question}
[요청]
이 생활 패턴에 맞는 카드를 추천해 줘.
'''
print(llm.invoke(base_prompt).content)

고객님의 생활 패턴에 적합한 카드로는 다음과 같은 카드를 추천해 드립니다.

1. 현대카드 The Classic
- 혜택: 외식 및 온라인 쇼핑 결제 시 캐시백 혜택 제공
- 조건: 월 소득 200만 원 이상, 연령 20세 이상
- 근거: 고객님의 외식 및 배달, 온라인 쇼핑 소비가 많으시기 때문에 캐시백 혜택을 통해 소비를 할인 받을 수 있습니다.

2. 삼성카드 The Select
- 혜택: 문화생활 및 레저 활동 결제 시 할인 혜택 제공
- 조건: 연간 카드 사용액 500만 원 이상, 연령 20세 이상
- 근거: 고객님의 취미 활동이 다양하고 문화생활을 즐기시는 것으로 보여, 문화생활 및 레저 활동 결제 시 할인 혜택을 받을 수 있는 이 카드를 추천드립니다.

3. KB국민카드 The Best
- 혜택: 온라인 쇼핑 및 영화, 콘서트 결제 시 포인트 적립 및 할인 혜택 제공
- 조건: 연간 카드 사용액 300만 원 이상, 연령 20세 이상
- 근거: 고객님의 온라인 쇼핑 소비가 많으시고 문화생활을 즐기시는 것으로 보여, 온라인 쇼핑 및 문화생활 결제 시 포인트 적립 및 할인 혜택을 받을 수 있는 이 카드를 추천드립니다.

이상으로 고객님의 생활 패턴에 맞는 카드를 추천해 드렸습니다. 부족한 부분이 있거나 더 궁금한 점이 있다면 언제든지 문의해 주세요. 감사합니다.


In [125]:
llm = ChatOpenAI(model = 'gpt-3.5-turbo',
                 temperature = 0.5,
                 api_key = MY_API_KEY
                )
base_prompt = f'''
너는 카드 추천 어시스턴트야.
사용자의 프로필과 요청에 맞춰 카드 3개를 추천하고,
각 카드의 근거(혜택-조건)를 설명해 줘.
[사용자 프로필]
{question}
[요청]
이 생활 패턴에 맞는 카드를 추천해 줘.
'''
print(llm.invoke(base_prompt).content)

고객님의 생활 패턴과 소비 습관을 고려하여 다음 3가지 카드를 추천해 드립니다.

1. 현대카드 The Classic
- 혜택: 외식 및 온라인 쇼핑 결제 시 캐시백 혜택 제공
- 조건: 월 소득 200만 원 이상, 신용등급 등급 A 이상

2. 삼성카드 The Traveler
- 혜택: 해외 여행 시 해외 이용료 면제 및 해외 호텔 할인 혜택 제공
- 조건: 월 소득 230만 원 이상, 해외 이용이 많은 경우 추천

3. KB국민카드 The Health
- 혜택: 영양제 및 운동 용품 구매 시 할인 혜택 제공
- 조건: 월 소득 200만 원 이상, 건강 관리에 관심이 많은 경우 추천

고객님의 취미활동 및 소비 습관을 고려하여 위 카드들을 추천해 드렸습니다. 혜택 및 조건을 참고하시어 적합한 카드를 선택해 보시기 바랍니다. 감사합니다.
